# Building a Spam Filter with Naive Bayes

The aim of this project is to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The program should classify new messages with an accuracy greater than 80% — more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, I will use dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.


### 1. Exploring the Dataset

We'll now start by reading in the data set.

In [1]:
import pandas as pd
sms = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, 
                  names = ['Label', 'SMS'])
sms.shape

(5572, 2)

In [2]:
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms.tail()

,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


Below, we can see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

In [4]:
sms['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

### 2. Splitting the data set to train and test sets

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [5]:
# Randomizing the dataset
sms_rand = sms.sample(frac = 1, random_state = 1)
sms_rand.head(5)

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:
# Calculataing the number of training SMS
train_no = round(len(sms_rand)*0.8)

# Separating the sms to train and test sets:
sms_train = sms_rand.iloc[:train_no].reset_index(drop = True)
sms_test = sms_rand.iloc[train_no:].reset_index(drop = True)

We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [7]:
sms_train['Label'].value_counts(normalize = True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [8]:
sms_test['Label'].value_counts(normalize = True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

### 3. Cleaning of the data sets

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)



We'll begin with removing all the punctuation and bringing every letter to lower case.

In [9]:
#Before cleaning
sms_train.head(5)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
sms_train['SMS'] = sms_train['SMS'].str.replace('\W', ' ', regex=True).str.lower().str.replace('\s+', ' ', regex=True)

In [11]:
# After cleaning
sms_train.head(5)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on da...


### 4. Creating the Vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [12]:
# Converting each string to a list
sms_train['SMS'] = sms_train['SMS'].str.split()
sms_train.head(5)

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [13]:
# Creating a list of unique worlds
list_train = []
for row in sms_train['SMS']:
    for word in row:
        if word in list_train:
            continue
        else:
            list_train.append(word)

There are 7,783 unique words in all the messages of our training set.

In [14]:
len(list_train)

7783

### 5. The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.

In [15]:
word_counts_per_sms = {unique_word: [0] * len(sms_train['SMS']) for unique_word in list_train}

for index, sms in enumerate(sms_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [16]:
# Transforming the dictionary to a DataFrame
train_word_df = pd.DataFrame(word_counts_per_sms)
train_word_df.head(3)

,yep,by,the,pretty,sculpture,yes,princess,are,you,going,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_df = pd.concat([sms_train, train_word_df], axis = 1)
full_train_df.head(3)

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Calculating Constants

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [18]:
# Calculating p_spam
p_spam = sum(full_train_df['Label'] == 'spam')/len(full_train_df)
p_spam

0.13458950201884254

In [19]:
# Calculating p_ham
p_ham = sum(full_train_df['Label'] == 'ham')/len(full_train_df)
p_ham

0.8654104979811574

In [20]:
# Calculating N_spam
n_words_per_spam_message = full_train_df[full_train_df['Label'] == 'spam']['SMS'].apply(len)
N_spam = n_words_per_spam_message.sum()

# Calculating N_ham
n_words_per_ham_message = full_train_df[full_train_df['Label'] == 'ham']['SMS'].apply(len)
N_ham = n_words_per_ham_message.sum()

# Calcultaind N_vocabulary
N_vocabulary = len(list_train)

# Setting alpha
alpha = 1

### 7. Calculating Parameters

Now we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [21]:
parameters_spam = {unique_word: 0 for unique_word in list_train}
parameters_ham = {unique_word: 0 for unique_word in list_train}

# Separating spam and ham sms
full_train_df_spam = full_train_df[full_train_df['Label'] == 'spam']
full_train_df_ham = full_train_df[full_train_df['Label'] == 'ham']

for words in list_train:
    parameters_spam[words] = ((full_train_df_spam[words].sum() + alpha)/
    (N_spam + alpha*N_vocabulary))
    parameters_ham[words] = ((full_train_df_ham[words].sum() + alpha)/
    (N_ham + alpha*N_vocabulary))
    

List of example parameters from spam:

In [22]:
list(parameters_spam.items())[:5]

[('yep', 4.3529360553693465e-05),
 ('by', 0.0015235276193792714),
 ('the', 0.006877638967483567),
 ('pretty', 4.3529360553693465e-05),
 ('sculpture', 4.3529360553693465e-05)]

### 8. Classifying A New Message

Now we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [23]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [24]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [25]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### 9. Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

Let's start by writing a function that returns classification labels instead of printing them.

In [26]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

We can use this function to create a new column in our test set.

In [27]:
sms_test['predicted'] = sms_test['SMS'].apply(classify_test_set)
sms_test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


We can measure the accuracy of the spam filter to find out how well does it work.

In [28]:
# Calculating the number of correct classifications
correct = 0
total = len(sms_test)
for rows in sms_test.iterrows():
    row = rows[1]
    if row['Label'] == row['predicted']:
        correct += 1


# Calculating accuracy
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. The spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

### Conclusion¶

In this project, I created a very accurate spam filter based on the multinomial Naive Bayes algorithm and a dataset of labeled 5,572 SMS. The spam filter takes in a new message and classifies it as spam or ham. The algorithm managed to reach an accuracy of 98.74%, which is almost 20% higher than our initial focus.